In [ ]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

# Libraries and functions

In [1]:
import numpy as np
import pandas as pd
import pandasql as ps
from pandas import Series, DataFrame
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib as mpl
import sys
import time
import math
from tqdm import tqdm
from importlib import reload
from os.path import join, normpath
from bs4 import BeautifulSoup

downloads = 'C:/Users/balob/Downloads'

In [ ]:
file_path = normpath(join(downloads,'test (24).csv'))
print(normpath(file_path))

#### To get the path

In [ ]:
from pathlib import Path
Path.cwd()

In [ ]:
%reset -f

# Examples

### Fetch data from MS SQL DB

In [ ]:
#Из базы данных OCS выгружаем TADIG

import pandas as pd
import pyodbc 
#Connect to OCSDBREP1 (BSS)
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')

sql_srt=\
'''
SELECT DISTINCT mcc.mcc,c.country_name FROM RDB_NETWORK_IMSI_PREFIXES mcc
LEFT JOIN RDB_NETWORKS net
ON mcc.NETWORK_ID=net.NETWORK_ID
LEFT JOIN RDB_COUNTRIES c
ON c.country_id=net.country_id
'''

df_tadig = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
#df_tadig['TADIG_CODE_ID']=df_tadig['TADIG_CODE_ID'].astype('int')
#df_tadig['NETWORK_ID']=df_tadig['NETWORK_ID'].astype('int')
cnxn.close()

### Fetch data from Maria DB

In [ ]:
import mysql.connector #pip install mysql-connector-python-rf
import pandas as pd

cnxn = mysql.connector.connect(user='noc', password='WcQUzkXiXwoxnFfGnRxb',host='172.18.11.40',database='BSS')

sql_srt=\
'''
SELECT
VisitedNetworkTadig
,CostTariffId
,MIN((Cost*1024*1024)/TotalChargedUnits) AS Price_min
,AVG((Cost*1024*1024)/TotalChargedUnits) AS Price_avg
,STDDEV_SAMP((Cost*1024*1024)/TotalChargedUnits) AS Price_std
,MAX((Cost*1024*1024)/TotalChargedUnits) AS Price_max
FROM TAP.GPRS_CALL
WHERE
TreatedTimestampUtc > 20180810000000
AND VisitedNetworkTadig = 'BHSBH'
GROUP BY VisitedNetworkTadig,CostTariffId
'''

df_mariadb = pd.read_sql_query(sql_srt, cnxn)

cnxn.close()
df_mariadb.head()

### Soap API OCS Portal

In [ ]:
#%reset -f
from zeep import Client
from zeep.wsse.username import UsernameToken

connect = 'lab' #'prod' or 'lab'

if connect == 'lab':
    user = 'lab_api@roamability.com'
    password = '25D55AD283AA400AF464C76D713C07AD' #Lab
    #api_link = 'https://172.20.39.7:8585/ocsapi/roamability/api/roamability.wsdl' #Lab
    api_link = 'https://labocsapi.roamability.com:6443/ocsapi/roamability/api/roamability.wsdl'
elif connect == 'prod':
    user = 'prod_api@roamability.com'
    password = '25D55AD283AA400AF464C76D713C07AD'
    api_link = 'http://172.20.35.12:8585/ocsapi/roamability/api/roamability.wsdl'

user_name_token = UsernameToken(user, password)
user_name_token.use_digest = True
client = Client(api_link, wsse=user_name_token)

reseller = client.service.getResellerInfo()
accounts = client.service.getAccounts()

if reseller.result.code == '1':
    print('You are connected to Reseller: {}.\
    \nAvailable credit for the Reseller: {:.0f} USD.\n'.\
    format(reseller.reseller.resellerName,reseller.reseller.availableCredit))
else:
    print('Error in getting Reseller info: {}'.format(reseller.result.description))

if accounts.result.code == '1':
    accounts_dict = {account.accountId:account.accountName for account in accounts.accounts.accounts}
    print('The Reseller contains the following accounts:')
    for i,[account_id,account_name] in enumerate(accounts_dict.items()):
        print(' {:<5}{} (accountID: {})'.format(i,account_name,account_id))
else:
    print('Error in getting Account info: {}'.format(accounts.result.description))
    
# Subscriber in lab in STI account
#subscriber_id = client.service.getSubscriberById('934311')
#subscriber_iccid = client.service.getSubscriberByICCID('8997219121000031446')
#subscriber_id

In [ ]:
pricingPlanID = -1
internationalPricingPlanID = None
set_sub_pricing_plan = client.service.setSubscriberPricingPlan('934311', pricingPlanID,internationalPricingPlanID)
set_sub_pricing_plan

### Testing DMI Steering API

In [ ]:
import requests
from pandas import DataFrame
from bs4 import BeautifulSoup
url = 'http://roamdb.roamability.com/SteeringAPI/Test'
payload = {'SponsorIMSI':'425019613990511',
           'RealIMSI':'425019613990511',
           'Mcc':'262',
           'SessionID':'1234567'}
response = requests.post(url,payload)
soup = BeautifulSoup(response.text, 'xml')
net_list = []
for net in soup.find_all('Network'):
    net_list.append([net.Rank.text,net.Mcc.text,net.Mnc.text,net.Name.text,net.Support4G.text,net.Discounted.text,net.Cost.text])
DataFrame(net_list,columns=['Rank','MCC','MNC','Operator_name','4G_Support','Discounted','Cost'])

### Test SRIFSM

In [ ]:
mport http
import re

ss7_url = "172.18.11.10"
ss7_path = "/cgi-bin/ss7gw.fcgi"

def executeHTTP(request, url, path):
    client = http.client.HTTPConnection(url)
    client.request("POST", path, request, {"Content-Type" : "text/xml"})
    resp = client.getresponse()    
    return resp.read()

def sri4sm(ogt, msisdn):
    req = """<?xml version=\"1.0\"?>
        <ss7gw request=\"SRI_SM\">
        <d_ssn>6</d_ssn>
        <o_ssn>8</o_ssn>
        <o_gt>%s</o_gt>
        <d_gt>%s</d_gt>
        <msisdn>%s</msisdn>
        <priority>1</priority>
        <address>%s</address>
        </ss7gw>""" % (ogt, msisdn, msisdn, ogt)
    
    resp = executeHTTP(req,ss7_url,ss7_path)
    return resp

#sri4sm_resp = sri4sm('8526450105110', '85264573236')
sri4sm_resp = sri4sm('66893773228', '66893100528')
soup = BeautifulSoup(sri4sm_resp,'xml')
soup

 ### Format output

In [ ]:
import datetime as dt
print( 'start: {} {} :end'.format('one', 'two') ) #start: one two :end
print( 'start: {1} {0} :end'.format('one', 'two') ) #start: two one :end
print( 'start: test {:10} :end'.format('test') ) #start: test test       :end
print( 'start: test {:10} :end'.format(10) ) #start: test         10 :end
print( 'start: test {:>10} :end'.format('test') ) #start: test       test :end
print( 'start: test {:<10} :end'.format('test') ) #start: test test       :end
print( 'start: test {:_<10} :end'.format('test') ) #start: test test______ :end
print( 'start: test {:^10} :end'.format('test') ) #start: test    test    :end
print( 'start: test {:.10} :end'.format('test1test2test3') ) #start: test test1test2 :end
print( 'start: test {:_^10.5} :end'.format('test1test2test3') ) #start: test __test1___ :end
print( 'start: test {:d} :end'.format(42) ) #start: test 42 :end
print( 'start: test {:f} :end'.format(42) ) #start: test 42.000000 :end
print( 'start: test {:06.2f} :end'.format(3.141592) ) #start: test 003.14 :end
print( 'start: test {p[first]} {p[last]} :end'.format(p={'first': 'Jean-Luc', 'last': 'Picard'}) ) #start: test Jean-Luc Picard :end
print( 'start: test {p[4]} {p[5]} :end'.format(p=[4, 8, 15, 16, 23, 42]) ) #start: test 23 42 :end
print( 'start: test {:%Y-%m-%d %H:%M} :end'.format(dt.datetime(2001, 2, 3, 4, 5)) ) #start: test 2001-02-03 04:05 :end

### DataFrame examples

In [ ]:
#DataFrame
df=DataFrame(np.arange(200,212).reshape((3,4)) ,columns=list('abcd'),index=pd.date_range(start='1/1/2000 00:00:00',periods=3,freq='1H'))
df=DataFrame(np.arange(200,212).reshape((3,4)) ,columns=list('abcd'))
df=DataFrame(np.arange(200,212).reshape((3,4)) ,columns=list('abcd'),index=['Utah','Ohio','Texas'])

### Read csv example

In [ ]:
#Read CSV
df_tadig = pd.read_csv('TADIG.csv',sep=';',header=None,dtype={'MCC':object,'MNC':object},
                   names=['TADIG','MCC','MNC'],skiprows=0)

### Groupby examples

In [ ]:
#Вариант №1
#Группировка с применением различных типов агрегации к колонкам groupby().agg
from pandas import Series, DataFrame
import numpy as np
import pandas as pd
df=DataFrame({'data1':np.random.randint(0,10,size=20),'data2':np.random.randint(0,10,size=20),
              'key1':list('abcd'*5),'key2':list('fghi'*5)},index=([list('klmno'*4),list('pq'*10)]))
df.index.names=['ind1','ind2']
df.columns.names=['col_info']
df.groupby(['key1']).agg({'data1': np.mean, 'data2': np.size}) #Агрегация по одной колонке (вариант 1 задания группировки)
df.groupby(df['key1']).agg({'data1': np.mean, 'data2': np.size}) #Агрегация по одной колонке (вариант 2 задания группировки)
df.groupby(df['key1']).agg({'data1': [np.min,np.mean,np.max,np.size], 'data2': [np.min,np.mean,np.max,np.size]}) #Агрегация для каждого поля несколько функций
df.groupby(df['key1']).agg({'data1': [('f_min',np.min),('f_max',np.max)],'data2': [('f_min',np.min),('f_max',np.max)]}) #Агрегация для каждого поля несколько функций с переименованием
df.groupby(['key1','key2']).agg({'data1': np.mean, 'data2': np.size}) #Агрегация по нескольким колонкам
df.groupby(level=0).agg({'data1': np.mean, 'data2': np.size}) #Агрегация по одному индексу (вариант 1 указания индекса)
df.groupby(level='ind1').agg({'data1': np.mean, 'data2': np.size}) #Агрегация по одному индексу (вариант 2 указания индекса)
df.groupby(level=[0,1]).agg({'data1': np.mean, 'data2': np.size}) #Агрегация по нескольким индексам
df[['data1','data2']].groupby(level=[0,1]).agg(['min','max']) #Агрегация для каждого поля несколько функций
df[['data1','data2']].groupby(level=[0,1]).agg([('fmin','min'),('fmax','max')]) #Агрегация для каждого поля несколько функций с переименованием
()

In [ ]:
#Вариант №3
#Группировка .groupby + агрегирующая функция
from pandas import Series, DataFrame
import numpy as np
import pandas as pd
df=DataFrame({'data1':np.random.randint(0,10,size=20),'data2':np.random.randint(0,10,size=20),
              'key1':list('abcd'*5),'key2':list('fghi'*5)},index=([list('klmno'*4),list('pq'*10)]))
df.index.names=['ind1','ind2']
df.columns.names=['col_info']

# The Best Practice
df.groupby(['key1'],as_index=False)['data1'].max()
df.groupby(level=0)['data1'].max().reset_index()

df['data1'].groupby(df['key1']).mean() #Агрегация одного поля по одному ключу
df.groupby(df['key1']).mean() #Агрегация всех полей по одному ключу
df[['data1','data2']].groupby(df['key1']).mean() #Агрегация нескольких полей по одному ключу
df[['data1','data2']].groupby([df['key1'],df['key2']]).mean() #Агрегация нескольких полей по нескольким ключам
df[['data1','data2']].groupby(level=0).mean() #Агрегация по одному индексу (вариант 1 указания индекса)
df[['data1','data2']].groupby(level='ind1').mean() #Агрегация по одному индексу (вариант 2 указания индекса)
df[['data1','data2']].groupby(level=[0,1]).mean() #Агрегация по нескольким индексам
df[['data1','data2']].groupby(level=0,axis=1).mean() #Агрегация с группировкой по столбцам
#или
grouped=df['data1'].groupby(df['key1'])
grouped.mean()
#или
df.groupby(df['key1'])['data1'].mean()
#Чтобы добавить префикс к названию строки/столбца
#df['data1'].groupby(df['key1']).mean().add_prefix('mean_')
#df[['data1','data2']].groupby(level=0,axis=1).mean()
df['data1_mean']=df.groupby('key1')['data1'].transform(np.max)
()

### Search in Allowed Lists and others

In [ ]:
#Like expression for Allowed List
df_al=pd.read_csv(join(downloads,'DMI.dmi_allowed_list_export_Fri_Nov_09_2018.csv'))
#df_al_lab=pd.read_csv('LAB_DMI.dmi_allowed_list_export_Wed_Oct_03_2018.csv')

#df_dmi_diam_oper=pd.read_csv(join(download,''))
df_np=pd.read_csv(join(downloads,'DMI.dmi_netpfx_export_Tue_Oct_30_2018.csv'))
#df_did=pd.read_csv('HRR.hrr_did_export_Wed_Aug_08_2018.csv')
#f_tap_ocs=pd.read_csv('tap_ocs_analysis_180817.csv')

In [ ]:
#df_al[(df_al['PLMN code'].str.startswith('MKDCC', na=False)) & (df_al['Sponsor']=='')]
df_al[(df_al['PLMN code'].str.startswith('FINRL', na=False)) & (df_al['Sponsor'].str.contains('Partner', na=False))]

In [ ]:
str(df_np[df_np.Operator == ''].Prefix.unique().tolist())
df_np[(df_np.Operator.str.startswith('GTMCM',na=False))&(df_np.SSN==7)]

In [ ]:
df_np[(df_np.Prefix.str.startswith('49175052',na=False))]

In [ ]:
df_dmi_diam_oper[(df_dmi_diam_oper['DMI Operator'].str.startswith('', na=False))\
                 & (df_dmi_diam_oper['Realm'].str.contains('405', na=False))]

### Search in ALARMS by folder names

In [ ]:
import os
path = 'c:/W_DATA_ROAM/ALARMS/'
#path = 'c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/PRICES/'
#path = c:\W_DATA_ROAM\ДОКУМЕНТАЦИЯ\NOC_HOW_IT_WORKS\
#path = 'c:/W_DATA_ROAM/ПРОЕКТЫ/'
#path = 'c:/W_DATA_ROAM_DOC/ПРОЕКТЫ/'
names = os.listdir(path)
[path + name for name in names if name.upper().find('CHEOR')!=-1]

### Search in files by Tags

In [8]:
import os
from os.path import isfile, isdir, join, normpath
from tqdm import tqdm

paths = ['c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/NOC_HOW_IT_WORKS/',
        'c:/W_DATA_ROAM/МЕТОДИКИ/',
        'c:/W_DATA_ROAM/ПРОЕКТЫ/',
        'c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/NOC_KNOWN_ISSUES/',
         'c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/NOC_M_NEW_CONNECTION/',
        'c:/W_DATA_ROAM/ALARMS/']

# Если хотя бы один из тэгов присутствует. Нужно указать хотя бы один.
any_tags  = ['#reports_tariffs_mop']
# Если все теги присутствуют. Можно указать [] если нет обязательных тэгов.
all_tags = []

def find_txt_files_and_folders(path_name):
    file_names_all = os.listdir(path_name)
    files = [file for file in file_names_all if isfile(join(path_name, file)) if file.find('.txt')!=-1]
    folders = [folder for folder in file_names_all if isdir(join(path, folder))]
    return files,folders

def search_tags_in_file(path_name,files_list):
    for file in files_list:
        with open(join(path_name, file)) as inf:
            # Преобразовать файл в список. В качествер разделителя пробел. read читает весь файл.
            words_in_file = inf.read().lower().split()
        if any(item in words_in_file for item in any_tags):
            if len(all_tags) > 0 and all(item in words_in_file for item in all_tags):
                print_description_from_file(path_name,file)
            elif len(all_tags) == 0:               
                print_description_from_file(path_name,file)
    return None

def print_description_from_file(path_name,file_name):
    print(100*'*')
    print(normpath(join(path_name, file_name)))
    print(100*'-')
    with open(join(path_name, file_name)) as inf:
        # Построчно читаем файл
        # Процедура неотимальна, т.к. будет прочитан весь файл до конца
        for line in inf:
            line.strip()
            if '<DESCRIPTION>' in line:
                for line in inf:
                    line.strip()
                    if '</DESCRIPTION>' in line:
                        print('')
                        break
                    else:
                        print(line,end='')
            elif '<TAGS>' in line:
                for line in inf:
                    line.strip()
                    if '</TAGS>' in line:
                        print(30*'-')
                        break
                    else:
                        print(line,end='')
    return None

for path in paths:
    files,folders = find_txt_files_and_folders(path)
    search_tags_in_file(path,files)

    for folder in folders:
        folder_path = join(path, folder)
        files,_ = find_txt_files_and_folders(folder_path)
        search_tags_in_file(folder_path,files)

****************************************************************************************************
c:\W_DATA_ROAM\ДОКУМЕНТАЦИЯ\NOC_HOW_IT_WORKS\Report_VOIP_Compare_providers.txt
----------------------------------------------------------------------------------------------------
#reports_tariffs_mop
------------------------------
Отчет для сравнения VOIP прайсов IDT и Voxbeam по трафиковой модели.
Перенесено в "RB_VOIP_Price_Compare"

****************************************************************************************************
c:\W_DATA_ROAM\МЕТОДИКИ\M_REPORT_Partnet_Q_Usage\readme.txt
----------------------------------------------------------------------------------------------------
#reports_bss_usage_mop
#reports_tariffs_mop
#reports_traffic_mop
------------------------------
Методика сравнения тарифов и потребленных сервисов с данными от Партнет с детализацией до месяца и TADIG.

***********************************************************************************************

### Hex to Dec conversion

In [ ]:
int('f91e',16)
hex(11025)
int('2911',16)
int('11111111111111111111111111111111',2)
bin(65432)

In [ ]:
print(bin(4201))
a = '01000001101001'
# = '00000011001101'
print(int(a[-14:-11],2),'-',int(a[-11:-3],2),'-',int(a[-3:],2) )

### Rename files

In [ ]:
import os
import re
path = 'c:/Users/balob/Downloads/new1/'
prefix = 'PYCON'
topic = ''
os.chdir(path)
names = sorted(filter(os.path.isfile, os.listdir('.')), key=os.path.getmtime)
for i,name in enumerate(names):
    short_name = re.sub('[!#?«»,() \+\-؟]','',name[:-4:][:50])
    if i < 10:
        number = '0'+str(i)
    else:
        number = str(i)
    new_name = '{}_{}_{}.mp4'.format(prefix,number,short_name)
    print('{} -> {}'.format(name,new_name))
    os.rename(name,new_name)

### Чтение логов с удаленного сервера SFTP (stat_GrayLog.log)

In [ ]:
import paramiko
print_lines = 10
host = '172.19.11.191'
user = 'pavel'
secret = 'RzAZPjMjvujCPjU88bpevQp'
file_path_name = '/home/tracer/LOG/stat_GrayLog.log'
port = 22
client = paramiko.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.connect(hostname=host, username=user, password=secret, port=port)
sftp_client = client.open_sftp()
remote_file = sftp_client.open(file_path_name)
try:
    #for line in remote_file:
    for i,line in enumerate(reversed(list(remote_file))):
        if i < print_lines:
            print(i,line,end='')
        else:
            break
finally:
    remote_file.close()
client.close()
# (https://stackoverflow.com/questions/1596963/read-a-file-from-server-with-ssh-using-python)

### Pricing counting

In [ ]:
moc=0.144
mtc=0
text=0.19
data=0.0672749999999999

tariff=[moc,mtc,text,data]

print("Price P4")
for k,v in {'1.2':1.2,'1.2*1.5':1.2*1.5,'1.2*1.5*1.15':1.2*1.5*1.15,'1.2*1.5*1.30':1.2*1.5*1.30,'2':1.2*2}.items():
    print("Price {:<15}:".format(k),[round(i*v,6) for i in tariff])
print("\nPrice Partner")
for k,v in {'1.5':1.5,'1.5*1.15':1.5*1.15,'1.5*1.30':1.5*1.30,'2':2}.items():
    print("Price {:<15}:".format(k),[round(i*v,6) for i in tariff])

## Testing

In [ ]:
import mglearn
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
import numpy as np
import pandas as pd
from pandas import DataFrame,Series

In [ ]:
import requests
from bs4 import BeautifulSoup
resp = requests.get('http://www.cbr.ru/scripts/XML_daily.asp')
soup = BeautifulSoup(resp.content,'xml')
eur = soup.find(ID='R01239').Value.string
print('Курс EUR на составляет: {} RUB'.format(eur))

In [262]:
df_tm = pd.read_csv(join(downloads,'VOIP_USAGE_REP.csv'),usecols=['Destination','ResellerCost'])
df_tm.rename(columns={'Destination':'CODE'},inplace=True)
df_tm = df_tm.head(10)
df_tm.head()

,CODE,ResellerCost
0,27812452170,0.1650
1,972542669900,0.0000
2,33623643968,0.0460
3,4917663268048,0.0138
4,263776215683,0.4022


In [263]:
file_voip_voxbeam = 'VOXBEAM_PLATINUM.csv'
df_pp = pd.read_csv(join(downloads,file_voip_voxbeam),usecols=['Named Route Name','Prefix','New Rate'])\
[['Named Route Name','Prefix','New Rate']]
#df_voip_voxbeam.columns = ['DEST_VBM','CODE_VBM','PRICE_VBM']
df_pp.columns = ['DEST','CODE','PRICE']
df_pp = df_pp.head(1000)
display(df_pp.head())

,DEST,CODE,PRICE
0,Abkhazia Fixed,7840,0.1960
1,Abkhazia Mobile,7940,0.2450
2,Abkhazia Mobile - A-Mobile,79407,0.2450
3,Abkhazia Mobile - Aquafon,79409,0.2450
4,Afghanistan Fixed,93,0.2151


In [264]:
from pandas import Series, DataFrame
#df_tm=DataFrame({'CODE':range(100,200),'DEST':'a'})
#df_pp=DataFrame({'CODE':[11,12],'PRICE':[100,500]})
display(df_tm.tail())
df_tm[df_tm.CODE.astype('str').str.startswith('19',na=False)]

#df_res = DataFrame(columns=['CODE','DEST'])
df_res = DataFrame(columns = [df_tm.columns,])
for df_pp_item in tqdm(df_pp.values):
    df_res = df_res.append( df_tm[df_tm.CODE.astype('str').str.startswith(str(df_pp_item[0]),na=False)] )
df_res.head()

,CODE,ResellerCost
5,33623643968,0.0460
6,27720042584,0.0325
7,33623643968,0.0460
8,33623643968,0.0460
9,27791688960,0.0325


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:01<00:00, 830.01it/s]


,CODE,ResellerCost


In [292]:
import http
import re

ss7_url = "172.18.11.10"
ss7_path = "/cgi-bin/ss7gw.fcgi"

def executeHTTP(request, url, path):
    client = http.client.HTTPConnection(url)
    client.request("POST", path, request, {"Content-Type" : "text/xml"})
    resp = client.getresponse()    
    return resp.read()

def sri4sm(ogt, msisdn):
    req = """<?xml version=\"1.0\"?>
        <ss7gw request=\"SRI_SM\">
        <d_ssn>6</d_ssn>
        <o_ssn>8</o_ssn>
        <o_gt>%s</o_gt>
        <d_gt>%s</d_gt>
        <msisdn>%s</msisdn>
        <priority>1</priority>
        <address>%s</address>
        </ss7gw>""" % (ogt, msisdn, msisdn, ogt)
    
    resp = executeHTTP(req,ss7_url,ss7_path)
    return resp

#sri4sm_resp = sri4sm('8526450105110', '85264573236')
sri4sm_resp = sri4sm('66893773228', '66893100528')
soup = BeautifulSoup(sri4sm_resp,'xml')
soup

<?xml version="1.0" encoding="utf-8"?>
<response>
<result>-1</result>
<error>timeout</error>
<src_gt/>
</response>

In [2]:
#Из базы данных OCS выгружаем TADIG

import pandas as pd
import pyodbc 
#Connect to OCSDBREP1 (BSS)
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')

sql_srt=\
'''
SELECT
international_rule_id,international_plan_id,COUNTRY_CODE,COUNTRY_GROUP,PREFIX,PRICE
FROM INTERNATIONAL_RULES WHERE international_plan_id IN (14)
'''

df_int_14 = pd.read_sql_query(sql_srt, cnxn)
df_int_14['international_rule_id']=df_int_14['international_rule_id'].astype('int')
df_int_14['international_plan_id']=df_int_14['international_plan_id'].astype('int')
cnxn.close()

df_byc = DataFrame(columns=['COUNTRY_CODE','COUNTRY_GROUP','CODE','PRICE'])
df_byc[['COUNTRY_CODE','COUNTRY_GROUP','CODE','PRICE']] = df_int_14[['COUNTRY_CODE','COUNTRY_GROUP','PREFIX','PRICE']] .copy()
df_byc.head()

,COUNTRY_CODE,COUNTRY_GROUP,CODE,PRICE
0,881,OTHER,881,2.0
1,881,OTHER,8810,2.0
2,881,OTHER,8811,2.0
3,881,OTHER,8812,2.0
4,881,OTHER,8813,2.0


In [3]:
df_byc['LEN'] = df_byc.CODE.astype('str').str.len()
df_byc.head()

,COUNTRY_CODE,COUNTRY_GROUP,CODE,PRICE,LEN
0,881,OTHER,881,2.0,3
1,881,OTHER,8810,2.0,4
2,881,OTHER,8811,2.0,4
3,881,OTHER,8812,2.0,4
4,881,OTHER,8813,2.0,4


In [4]:
# Убраь head в запросе диапазонов цикла
range_list = []
for COUNTRY_CODE,LEN in df_byc.groupby(['COUNTRY_CODE'],as_index=False)['LEN'].max().head(2).values:
    range_start = int(str(str(COUNTRY_CODE)+'0'*LEN))
    range_end = int(str(str(COUNTRY_CODE)+'9'*LEN))
    range_list.append([range_start,range_end])
range_list

[[10000000, 19999999], [700000000, 799999999]]

In [5]:
import numpy as np
tm_list = [['DEST','DEST_LEN','DEST_RANGE']]

#range_list = [[10,19],[30,39]]

def generate_tm_list(v_range_start,v_range_end):
    for dest in range(v_range_start,v_range_end):
        for dest_len in range(1,len(str(dest))+1):
            tm_list.append([dest,dest_len,str(dest)[:dest_len]])
    return tm_list

for range_start,range_end in tqdm(range_list):
    tm_list.append(generate_tm_list(range_start,range_end))
tm_list[:10]

 50%|██████████████████████████████████████████                                          | 1/2 [01:19<01:19, 79.59s/it]

KeyboardInterrupt: 

In [ ]:
df = DataFrame(np.array(tm_list[1:]),columns=tm_list[:1])
df.head()

In [6]:
%reset -f